<a href="https://colab.research.google.com/github/harsha-bsm/Topic-Modelling-using-LDA-and-NMF-on-SamsungM30s-reviewsflipkart/blob/master/POS_TAGGING_only_NOUNS_LDA_SAMSUNGM30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import spacy
import pandas as pd

In [ ]:
data=pd.read_csv("samsungdata.csv")
dataoriginal=pd.read_csv("samsungdata.csv")

In [ ]:
pip install emot 

In [ ]:
####### Apply emoticons and emojis

from emot.emo_unicode import UNICODE_EMO, EMOTICONS # contains english text of emojis and emoticons in dict format 
def convert_emojis(review):
  for x in review:
    if x in UNICODE_EMO.keys():
      review=review.replace(x,"")
  return review

In [ ]:
#converting all basic contracted foms  
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
#removing special characters and tabs ,spaces,new lines etc..
def removecharacters(review):
  review= re.sub('[^A-Za-z0-9]+',' ',review)  #anything exept numbers and alphabets, replace them with space
  review=re.sub(r"\n"," ",review)  #new lines into space
  review=re.sub(r"\t"," ",review)  #tabs into space
  review=re.sub(r"\v"," ",review)  #vertical tab into space
  review=re.sub(r"\s"," ",review)   #all extra spaces into single space
  return review  


In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step
# ignore words like won't, haven't etc.. We have already taken care of them

stopword= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not","phone","product","mobile","np"])

In [ ]:
from nltk.corpus import stopwords
import nltk

In [ ]:
def removestopwords(review):
  return " ".join([word for word in review.split(" ") if not word in stopword])


In [ ]:
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:removestopwords(x)))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import ldamodel
import gensim.corpora
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
#https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(model, feature_names, n_top_words):
  topics_cluster=[]
  for topic_idx, topic in enumerate(model.components_):
      message = "Topic #%d: " % topic_idx
      message += ",".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
      topics_cluster.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
  return topics_cluster
  

## POS TAGGING taking only nouns

In [ ]:
  import nltk
  nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import spacy

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
def only_nouns(review):
  lis= " ".join([each.strip(" ") for each,pos in nltk.pos_tag(review.split()) if pos=="NN"])
  return lis
  


In [ ]:
data["review"]=data["review"].apply(lambda x:only_nouns(x))

### LDA BOW ---  (topipc extraction/keyword extraction)

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),20)

[['price',
  'battery',
  'quality',
  'range',
  'bit',
  'buy',
  'mark',
  'camera',
  'performance',
  'power',
  'network',
  'experience',
  'button',
  'voice',
  'finger',
  'print',
  'lagsnetwork',
  'month',
  'amazingperformance',
  'none'],
 ['battery',
  'camera',
  'backup',
  'performance',
  'budget',
  'smartphone',
  'handset',
  'hand',
  'range',
  'experience',
  'gaming',
  'display',
  'samsung',
  'good',
  'bokeh',
  'features1',
  'phonelet',
  'charger3',
  'dispay',
  'phon'],
 ['superb',
  'samsung',
  'flipkart',
  'delivery',
  'm30s',
  'super',
  'galaxy',
  'backup',
  'camara',
  'selfie',
  'bugdet',
  'mar',
  'gui',
  'priceworthy',
  'beatty',
  'oshawa',
  'service',
  'effort',
  'thks',
  'device'],
 ['battery',
  'wesome',
  'display',
  'camera',
  'life',
  'mah',
  'money',
  'isplay',
  'hand',
  'love',
  'glass',
  'use',
  'package',
  'rupee',
  'ram',
  'drain',
  'work',
  'day',
  'management',
  'feel'],
 ['super',
  'use',
  'xce

In [ ]:
topics=print_top_words(lda,bow.get_feature_names(),6)
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,dominant_topic,review,dominant_topic_keywords
0,0.006667,0.939996,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006668,0.006668,1,Bokeh mode is very excellent than any other ha...,"[battery, camera, backup, performance, budget,..."
1,0.909989,0.010004,0.010001,0.010001,0.010001,0.010001,0.010000,0.010000,0.010001,0.010002,0,Just awsome product the picture quality is goo...,"[price, battery, quality, range, bit, buy]"
2,0.008334,0.008336,0.924983,0.008335,0.008335,0.008334,0.008333,0.008333,0.008337,0.008339,2,Samsung galaxy m30s is most useful.But Samsung...,"[superb, samsung, flipkart, delivery, m30s, su..."
3,0.007143,0.007143,0.007144,0.007143,0.007143,0.935712,0.007143,0.007143,0.007143,0.007143,5,Not worth for me. i buy for my samsung tv this...,"[online, buy, call, function, time, purchase]"
4,0.008334,0.924997,0.008333,0.008335,0.008333,0.008333,0.008333,0.008333,0.008334,0.008334,1,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[battery, camera, backup, performance, budget,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,0.050000,0.050000,0.050000,4,okay,"[super, use, xcellent, performance, experience..."
294,0.033333,0.033337,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.699996,0.033333,8,Super style and good look nice product,"[display, battery, camera, problem, ok, price]"
295,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0,good,"[price, battery, quality, range, bit, buy]"
296,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0,good,"[price, battery, quality, range, bit, buy]"


In [ ]:
modelcomponents=lda.components_/(np.sum(lda.components_,axis=1)).reshape(-1,1)
pd.DataFrame(modelcomponents,index=topic,columns=bow.get_feature_names())

,ability,amazing,amazingperformance,amoled4,android,anything,apple,awesome,awsm,awsome,backup,battery,battry,beast,beatty,bettery,bit,body,bokeh,box,bro,budget,bugdet,butter,button,buy,call,cam,camara,camera,capture,cast,cell,chalta,charge,charger,charger3,choic,choice,coin,...,sm,smart,smartphone,software,sooooooperb,speaker,stabilization,state,style,super,superb,superrrrbbbbb,suprrrrrrrrr,switch,task,team,term,thakarthu,thank,thing,thks,till,time,touch,tq,tunney,tv,use,value,verall,video,voice,waysome,wesome,wid,work,worth,worthless,wsm,xcellent
topic_0,0.013302,0.001209,0.013302,0.001209,0.001209,0.001209,0.001209,0.001209,0.013302,0.013302,0.013302,0.040337,0.001209,0.001209,0.001209,0.001209,0.025395,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.013302,0.025395,0.001209,0.001209,0.001209,0.025393,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,...,0.013302,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.013302,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.013301,0.001209,0.001209,0.001209,0.013302,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209
topic_1,0.000853,0.009385,0.000853,0.009385,0.000853,0.009385,0.000853,0.000853,0.000853,0.000853,0.051221,0.133712,0.009385,0.009385,0.000853,0.000853,0.000853,0.000853,0.009385,0.000853,0.000853,0.026450,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.071924,0.000853,0.000853,0.000853,0.009385,0.009385,0.000853,0.009385,0.009385,0.000853,0.000853,...,0.000853,0.000853,0.026450,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.009385,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.009385,0.000853,0.009385,0.000853,0.000853,0.000853,0.009385,0.000853,0.000853,0.000853,0.009385,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853
topic_2,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.016506,0.001379,0.001379,0.001379,0.015173,0.001379,0.001379,0.001379,0.001379,0.001379,0.015173,0.001379,0.015173,0.001379,0.001379,0.001379,0.001379,0.001379,0.015173,0.015172,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,...,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001380,0.001379,0.028967,0.125522,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.015173,0.001379,0.001379,0.001379,0.001382,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379
topic_3,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.092947,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.035566,0.012247,0.001113,0.012247,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,...,0.001113,0.012247,0.001113,0.012247,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.012247,0.001113,0.001113,0.012247,0.001113,0.001113,0.001113,0.001113,0.012247,0.001113,0.001113,0.001113,0.001113,0.001113,0.090184,0.001113,0.012247,0.012245,0.001113,0.001113,0.001113
topic_4,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.015625,0.001420,0.001420,0.001420,0.015625,0.001420,0.001420,0.001420,0.001420,0.015625,0.001420,0.001420,0.001420,0.001420,0.001420,0.015625,0.001420,0.001420,0.001420,0.001420,0.001420,0.015625,0.001420,0.001421,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,...,0.001420,0.001420,0.015625,0.001420,0.015625,0.015625,0.001420,0.001420,0.001420,0.086648,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.

#### (LDA again - on topics obtained) 

In [ ]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['battery', 'camera', 'delivery', 'online'],
 ['performance', 'excellent', 'problem', 'osm'],
 ['bit', 'nice', 'thank', 'value'],
 ['quality', 'display', 'box', 'backup']]

In [ ]:
topics=print_top_words(lda,bow.get_feature_names(),4)
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,dominant_topic,review,dominant_topic_keywords
0,0.125119,0.125211,0.624471,0.125199,2,Bokeh mode is very excellent than any other ha...,"[bit, nice, thank, value]"
1,0.624862,0.125050,0.125040,0.125047,0,Just awsome product the picture quality is goo...,"[battery, camera, delivery, online]"
2,0.125046,0.125081,0.125065,0.624809,3,Samsung galaxy m30s is most useful.But Samsung...,"[quality, display, box, backup]"
3,0.125102,0.125181,0.125146,0.624572,3,Not worth for me. i buy for my samsung tv this...,"[quality, display, box, backup]"
4,0.125119,0.125211,0.624471,0.125199,2,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[bit, nice, thank, value]"
5,0.624862,0.125050,0.125040,0.125047,0,I have realized that this phone is full packag...,"[battery, camera, delivery, online]"
6,0.624826,0.125063,0.125051,0.125060,0,"one of the good phone in this price range , th...","[battery, camera, delivery, online]"
7,0.125102,0.125181,0.125146,0.624572,3,Nice mobile and it comes good battery back up....,"[quality, display, box, backup]"
8,0.125028,0.624887,0.125039,0.125046,1,Been using this phone for more than 4 months n...,"[performance, excellent, problem, osm]"
9,0.125119,0.125211,0.624471,0.125199,2,This phone is works excellent so far after (6 ...,"[bit, nice, thank, value]"


### LDA TFIDF

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=8,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,4)

[['awesome', 'excellent', 'isplay', 'sm'],
 ['camera', 'super', 'quality', 'm30s'],
 ['superb', 'samsung', 'delivery', 'love'],
 ['battery', 'price', 'wesome', 'backup'],
 ['ok', 'smartphone', 'use', 'purchase'],
 ['money', 'value', 'call', 'delivery'],
 ['nice', 'experience', 'function', 'thank'],
 ['ear', 'perfomance', 'android', 'butter']]

#### (LDA again - on topics obtained) 

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['super', 'ear', 'glass', 'butter'],
 ['delivery', 'm30s', 'use', 'rattan'],
 ['camera', 'display', 'isplay', 'wesome'],
 ['experience', 'ok', 'thank', 'mi']]

### NMF BOW (better results)

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])


In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])

nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,5)

[['phone', 'superb', 'use', 'range', 'battery'],
 ['product', 'delivery', 'buy', 'function', 'team'],
 ['battery', 'display', 'camera', 'price', 'performance'],
 ['mobile', 'samsung', 'super', 'performance', 'flipkart'],
 ['camera', 'quality', 'money', 'wesome', 'value']]

In [ ]:
topics=print_top_words(nmf,bow_words,5)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(np.round(nmf.transform(bow_review),2),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,topic_4,dominant_topic,review,dominant_topic_keywords
0,0.00,0.00,1.14,0.00,0.0,2,Bokeh mode is very excellent than any other ha...,"[battery, display, camera, price, performance]"
1,0.21,0.57,0.10,0.00,0.3,1,Just awsome product the picture quality is goo...,"[product, delivery, buy, function, team]"
2,0.18,0.00,0.17,0.70,0.0,3,Samsung galaxy m30s is most useful.But Samsung...,"[mobile, samsung, super, performance, flipkart]"
3,0.00,1.52,0.00,0.23,0.0,1,Not worth for me. i buy for my samsung tv this...,"[product, delivery, buy, function, team]"
4,0.09,0.00,0.90,0.00,0.0,2,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[battery, display, camera, price, performance]"
...,...,...,...,...,...,...,...,...
293,0.00,0.00,0.00,0.00,0.0,0,okay,"[phone, superb, use, range, battery]"
294,0.00,0.34,0.00,0.00,0.0,1,Super style and good look nice product,"[product, delivery, buy, function, team]"
295,0.00,0.00,0.00,0.00,0.0,0,good,"[phone, superb, use, range, battery]"
296,0.00,0.00,0.00,0.00,0.0,0,good,"[phone, superb, use, range, battery]"


#### (NMF again - on topics obtained) 

In [ ]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['camera', 'cost', 'delivery', 'mark'],
 ['life', 'voice', 'wesome', 'backup'],
 ['performance', 'awesome', 'quality', 'ok'],
 ['battery', 'buy', 'flipkart', 'display'],
 ['xcellent', 'price', 'awsm', 'box']]

### NMF TFIDF

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()

In [ ]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,4)

[['battery', 'backup', 'camera', 'performance'],
 ['camera', 'quality', 'experience', 'light'],
 ['wesome', 'life', 'model', 'battery'],
 ['price', 'range', 'segment', 'mid'],
 ['super', 'flipkart', 'purchase', 'use'],
 ['superb', 'delivery', 'condition', 'galaxy'],
 ['money', 'value', 'time', 'delivery'],
 ['samsung', 'love', 'm30s', 'performance'],
 ['display', 'ok', 'xcellent', 'smartphone'],
 ['nice', 'awesome', 'osm', 'mi']]

In [ ]:
topics=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(nmf.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,dominant_topic,review,dominant_topic_keywords
0,0.0,0.000000,0.0,0.000000,0.599101,0.0,0.000000,0.0,0.000000,0.0,4,Bokeh mode is very excellent than any other ha...,topic_4
1,0.0,0.761225,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,1,Just awsome product the picture quality is goo...,topic_1
2,0.0,0.000000,0.0,0.763039,0.000000,0.0,0.000000,0.0,0.000000,0.0,3,Samsung galaxy m30s is most useful.But Samsung...,topic_3
3,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.382495,0.0,8,Not worth for me. i buy for my samsung tv this...,topic_8
4,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.329293,0.0,0.000000,0.0,6,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...",topic_6


#### (NMF again - on topics obtained) ---------------------------------------------------

In [ ]:
arr=np.array(print_top_words(nmf,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['camera', 'model', 'money', 'smartphone'],
 ['performance', 'm30s', 'time', 'nice'],
 ['delivery', 'keybad', 'galaxy', 'ok'],
 ['battery', 'xcellent', 'quality', 'backup']]